In [12]:
# ls

In [17]:
# !git clone https://github.com/Reclist/reclist/
# !cd reclist ; pip install -e .

In [3]:
# !git clone https://github.com/RecList/evalRS-KDD-2023
# !cd evalRS-KDD-2023 ; git checkout readme_kdd_2023

In [1]:
# !which python
# !pip install -r evalRS-KDD-2023/requirements.txt
# !cd reclist

In [13]:
import os
import sys
sys.path.insert(0, 'evalRS-KDD-2023/evaluation')

In [14]:
import gc
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow.parquet as pq

# sns.set_theme("notebook")
# Change notebook data format to be like 100.10 instead of scientific notation
pd.options.display.float_format = '{:.2f}'.format

In [26]:
from gensim.models import Word2Vec, KeyedVectors

In [7]:
# !cd reclist
# !pip install -e .

In [92]:
# !pip install reclist

In [15]:
from EvalRSRunner import ChallengeDataset

In [18]:
# import os
# import sys
# sys.path.append("evalRS-KDD-2023/evaluation")
# from EvalRSRunner import ChallengeDataset
# from EvalRSReclist import EvalRSReclist
# from reclist.reclist import LOGGER, METADATA_STORE
# from gensim.models import KeyedVectors
# import pandas as pd
# import numpy as np

In [17]:
# !pip install reclist

In [19]:
dataset = ChallengeDataset(force_download=False)

LFM dataset already downloaded. Skipping download.
Loading dataset.
Generating Train/Test Split.
Generating dataset hashes.


In [24]:
train, test = dataset.get_sample_train_test()
train.shape, test.shape

# train['train'] = 1
# test['train'] = 0

((6869679, 6), (29722, 3))

In [30]:
# # No point combining cause time stamp information is null so irrelevant and all the test users are present in train set. So just use customerN from train df

# convert the timestamp in a more convenient format
train["timestamp"] = pd.to_datetime(train["timestamp"], unit="s")

# A session id per customer based on a 6 hr window, if their last session was more than 6 hrs ago, then it is a new session
train['session_id'] = train.groupby(['user_id', pd.Grouper(key='timestamp', freq='6H')]).ngroup()

In [33]:
user_session_df = train.groupby(['user_id', 'session_id'])['track_id'].count().reset_index()
user_session_df.columns = ['user_id', 'session_id', 'tracks_listened']


user_summary = user_session_df.groupby('user_id').agg({'tracks_listened': np.median, 'session_id': 'nunique'})
user_summary = user_summary.reset_index()
user_summary.columns = ['user_id', 'customern_tracks', 'session_count']
user_summary['customern_tracks'] = user_summary['customern_tracks'].apply(int)

In [34]:
user_summary.head()

,user_id,customern_tracks,session_count
0,384,3,47
1,10879,6,29
2,12619,2,38
3,14308,3,39
4,15067,8,31


In [5]:
class EvalRSSimpleModel:
    """
    This is a dummy model that returns random predictions on the EvalRS dataset.
    """
    def __init__(self, items: pd.DataFrame, top_k: int=10, **kwargs):
        self.items = items
        self.top_k = top_k
        print("Received additional arguments: {}".format(kwargs))

    def predict(self, user_ids: pd.DataFrame) -> pd.DataFrame:
        k = self.top_k
        num_users = len(user_ids)
        pred = self.items.sample(n=k*num_users, replace=True).index.values
        pred = pred.reshape(num_users, k)
        pred = np.concatenate((user_ids[['user_id']].values, pred), axis=1)
        return pd.DataFrame(pred, columns=['user_id', *[str(i) for i in range(k)]]).set_index('user_id')

In [6]:
# dummy model
my_df_model = EvalRSSimpleModel(dataset.df_tracks, top_k=100)
# get some predictions
df_predictions = my_df_model.predict(dataset._test_set[['user_id']])

Received additional arguments: {}


In [35]:
df_predictions.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
user_id,,,,,,,,,,,,,,,,,,,,,
384,9726652,1904440,259175,29477086,31367,946816,1475219,476705,1077625,2075310,...,285156,2534828,29568301,279435,43438,265754,17716897,3547175,2296164,2369852
10879,1167440,257125,2005050,2140138,3564380,3183366,422202,3761697,94227,660169,...,30713646,993042,7785713,583202,582200,30149499,510261,2402779,325125,40333
12619,2992251,2326649,583800,1910386,2783739,335488,6345055,945144,465046,686549,...,631897,19576,3797817,743402,1755231,608758,29446802,3562272,2890121,1466287
14308,1312886,799433,345316,256314,1914377,2775444,7119685,1294781,2576432,3835573,...,2942912,2013887,20815588,1368696,276960,1980616,61735,262894,308338,349726
15067,745534,19692825,871606,1257413,2863835,1784274,1370151,2141468,692635,4276093,...,641154,789155,625017,485120,845513,2619451,483849,777193,1076182,491287


In [44]:
df_predictions = df_predictions.reset_index()

In [52]:
df_predictions.columns[1:]

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60',
       '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72',
       '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84',
       '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96',
       '97', '98', '99'],
      dtype='object')

In [54]:
# Convert from wide to long format, with user_id and impression_pos as index. Current data is columns: user_id, 0, 1, 2, 3...
# It finally needs to be user_id, impression_pos, impression_id (currently cell value)

df_predictions_long = pd.melt(df_predictions, id_vars='user_id', value_vars=df_predictions.columns[1:])
df_predictions_long.columns = ['user_id', 'impression_pos', 'impression_id']

In [55]:
df_predictions_long.head()

,user_id,impression_pos,impression_id
0,384,0,9726652
1,10879,0,1167440
2,12619,0,2992251
3,14308,0,1312886
4,15067,0,745534


In [56]:
df_predictions_long[df_predictions_long['user_id']==384]

,user_id,impression_pos,impression_id
0,384,0,9726652
29722,384,1,1904440
59444,384,2,259175
89166,384,3,29477086
118888,384,4,31367
...,...,...,...
2823590,384,95,265754
2853312,384,96,17716897
2883034,384,97,3547175
2912756,384,98,2296164


In [59]:
# join with user_summary df
df_prediction_cn = df_predictions_long.merge(user_summary, how='inner', on = ['user_id'])

In [61]:
df_prediction_cn.head()

,user_id,impression_pos,impression_id,customern_tracks,session_count
0,384,0,9726652,3,47
1,384,1,1904440,3,47
2,384,2,259175,3,47
3,384,3,29477086,3,47
4,384,4,31367,3,47


In [64]:
df_prediction_cn['impression_pos'] = df_prediction_cn['impression_pos'].astype(int)

In [65]:

df_prediction_cn = df_prediction_cn[df_prediction_cn['impression_pos'] <= df_prediction_cn['customern_tracks']-1]

In [66]:
df_prediction_cn.head()

,user_id,impression_pos,impression_id,customern_tracks,session_count
0,384,0,9726652,3,47
1,384,1,1904440,3,47
2,384,2,259175,3,47
100,10879,0,1167440,6,29
101,10879,1,257125,6,29


In [70]:
df_prediction_cn = df_prediction_cn[['user_id', 'impression_pos', 'impression_id']]

In [71]:
df_prediction_cn

,user_id,impression_pos,impression_id
0,384,0,9726652
1,384,1,1904440
2,384,2,259175
100,10879,0,1167440
101,10879,1,257125
...,...,...,...
2972101,50967444,1,2486655
2972102,50967444,2,5611462
2972103,50967444,3,29483554
2972104,50967444,4,97334


In [72]:
df_prediction_cn['impression_pos'] = df_prediction_cn['impression_pos'].astype(str)

/var/folders/s0/bcv3d1_d5pq2s0z0zp0bw2v00000gp/T/ipykernel_12290/1384590851.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prediction_cn['impression_pos'] = df_prediction_cn['impression_pos'].astype(str)


In [73]:
df_prediction_cn.head()

,user_id,impression_pos,impression_id
0,384,0,9726652
1,384,1,1904440
2,384,2,259175
100,10879,0,1167440
101,10879,1,257125


In [75]:
df_prediction_cn = df_prediction_cn.pivot(index='user_id', columns='impression_pos', values='impression_id')

In [11]:
df_prediction_cn

NameError: name 'df_prediction_cn' is not defined

In [76]:
# initialize with everything

# load a similarity model: here we used
similarity_model = KeyedVectors.load(os.path.join(dataset.path_to_dataset, 'song2vec.wv'))

# initialize with everything
cdf = EvalRSReclist(
    dataset=dataset,
    model_name="SimpleModel",
    predictions=df_predictions,
    logger=LOGGER.LOCAL,
    metadata_store=METADATA_STORE.LOCAL,
    similarity_model=similarity_model,
)

# run reclist
cdf(verbose=True)

NameError: name 'KeyedVectors' is not defined

## CBOW model

In [20]:
class CBOWRecSys:

    def __init__(self, top_k: int=100):
        super(CBOWRecSys, self).__init__()
        """
        :param top_k: numbers of recommendation to return for each user. Defaults to 20.
        """
        self.top_k = top_k
        self.mappings = None

    def train(self, train_df: pd.DataFrame, **kwargs):
        print("Training model")

        # let's put tracks in order
        df = train_df[['user_id', 'track_id', 'timestamp']].sort_values('timestamp')

        # we group by user and create sequences of tracks
        p = df.groupby('user_id', sort=False)['track_id'].agg(list)

        # we now build "sentences" : sequences of tracks
        sentences = p.values.tolist()

        # train word2vec:
        # large window and small vector size...lots of epochs
        self.mymodel = Word2Vec(sentences, min_count=0, vector_size=20, window=40, epochs=5)

        user_tracks = pd.DataFrame(p)

        # we sample 10 songs for each user
        user_tracks["track_id_sampled"] = user_tracks["track_id"].apply(lambda x : random.choices(x, k=40))

        self.mappings = user_tracks.T.to_dict()

    def predict(self, user_ids: pd.DataFrame) -> pd.DataFrame:
        """

        This function takes as input all the users that we want to predict the top-k items for, and
        returns all the predicted songs.

        While in this example is just a random generator, the same logic in your implementation
        would allow for batch predictions of all the target data points.

        """
        user_ids = user_ids.copy()
        k = self.top_k


        pbar = tqdm(total=len(user_ids), position=0)

        predictions = []
        for user in user_ids["user_id"]:
            user_tracks = self.mappings[user]["track_id_sampled"]
            get_user_embedding = np.mean([self.mymodel.wv[t] for t in user_tracks], axis=0)

            # we need to filter out stuff from the user history. We don't want to suggest to the user something they
            # have already listened to
            max_number_of_returned_items = len(self.mappings[user]["track_id"]) + self.top_k

            user_predictions = [k[0] for k in self.mymodel.wv.most_similar(positive=[get_user_embedding], topn=max_number_of_returned_items)]
            user_predictions = list(filter(lambda x: x not in self.mappings[user]["track_id"], user_predictions))[0:self.top_k]

            predictions.append(user_predictions)

            pbar.update(1)
        pbar.close()

        # lil trick to reconstruct a dataframe that has user ids as index and the predictions as columns
        # This is a very important part! consistency in how you return the results is fundamental for the evaluation

        users = user_ids["user_id"].values.reshape(-1, 1)
        predictions = np.concatenate([users, np.array(predictions)], axis=1)
        return pd.DataFrame(predictions, columns=['user_id', *[str(i) for i in range(k)]]).set_index('user_id')

In [21]:
# instantiate this
cbow_recsys = CBOWRecSys()

In [27]:
%%time
cbow_recsys.train(train)

Training model


NameError: name 'random' is not defined

In [ ]:
%%time
predicted = cbow_recsys.predict(test) # let's test only on a couple of elements